# **Checking The Address Table**
상권분할 작업의 내용수정 및 보완작업

## **1 Loading the table**
엑셀 원본데이터 불러오기

In [1]:
import pandas as pd
file_excel = './data/store.xlsx'
table_excel = pd.read_excel(file_excel)
test_table = table_excel[table_excel['상호명']=='24시셀프빨래방']
test_table

,연번,상호명,점주명,연락처,주소,업종,행정동,사진,임대차,비고,Unnamed: 10
38,39,24시셀프빨래방,민혜경,010-4196-7646,"관악로 118, 1층",빨래방,난곡동,O,O,개별점포,NaN


## **2 데이터 유효성 검사**
Naver Map API 를 활용하여 결과값 확인하기
- **[NCloud 설명서](https://apidocs.ncloud.com/ko/ai-naver/maps_static_map/)**
- **[Naver MAP Tutorial](https://navermaps.github.io/maps.js.ncp/docs/tutorial-1-Conceptual-Overview.html)**

In [2]:
from data.arterior import Arterior
test_result = Arterior().naverAddress(list(test_table['주소'].values)[0])
test_token  = list(test_table['행정동'].values)[0]
if test_result[0].find(test_token) == -1:
    print(f'Address is Not Matched !!!\n"{test_token}" =\= {test_result[1]}')

Address is Not Matched !!!
"난곡동" =\= 서울특별시 관악구 봉천동 1596-8


<br/>

# **Create the ERD DataBase Table**
상권분할 작업의 내용수정 및 보완작업

## **1 Loading and Split the Table**
테이블 자료를 사람과 주소 내용으로 구분하기
```
table = table[(table['view']==1) & (table['fix']==1)]
```

In [3]:
from collections import Counter
artist_code  = {}
file_name    = "./data/artstore_match_v3.csv"
table_raw    = pd.read_csv(file_name).fillna('')
table_artist = table_raw.iloc[:,:3].drop_duplicates().reset_index(drop=True)
for _ in range(len(table_artist)):
    table_series = table_artist.loc[_,:]
    artist_code[table_series['artist']] = f"{table_series['group']}-{table_series['artist_no']}"

# table 유효값 필터링 및 내용의 확인
table = table_raw[table_raw['view']==1].reset_index(drop=True)
for _ in ['A','B','C']:
    temp_table = table[table['group']==_]
    print(f'{_} :', dict(Counter(temp_table['artist_no'].to_list())))

A : {1: 4, 2: 4, 3: 4, 4: 4, 5: 4, 6: 4, 7: 4, 8: 4, 9: 4, 10: 4, 11: 4, 12: 4, 13: 4}
B : {1: 4, 2: 4, 3: 4, 4: 4, 5: 4, 6: 4, 7: 4, 8: 4, 9: 4, 10: 4, 11: 4, 12: 4}
C : {1: 4, 2: 4, 3: 4, 4: 4, 5: 4, 6: 4, 7: 4, 8: 4, 9: 4, 10: 4, 11: 4, 12: 4}


In [4]:
artist_list = [[k] + v.split('-')  for k,v in artist_code.items()]
artist_table = pd.DataFrame(artist_list)
artist_table.columns = ['name', 'group', 'no']
artist_table.insert(3, 'etc', '')
artist_table.insert(3, 'homepage', '')
artist_table.insert(3, 'instagram', '')
artist_table.insert(3, 'email', '')
artist_table.insert(3, 'phone', '')
artist_table.tail(2)

,name,group,no,phone,email,instagram,homepage,etc
38,나혜지,M,2,,,,,
39,정효윤,M,3,,,,,


## **2 Insert the Data**
자료 추가하기

In [5]:
data_add = [
    ['전은미','010-4471-5995','multieunmi@naver.com'],
    ['나혜지','010-5553-8404','thedesignlab@naver.com'],
    ['정효윤','010-8200-7607','hony0709@naver.com']
]
# 데이터 추가는 loc, iloc 를 활용하기
for _ in data_add:
    index_number = list(artist_table[artist_table['name']== _[0] ].index)[0]
    for no, columname in enumerate(['phone', 'email']):
        artist_table.loc[index_number,columname] = _[no+1]
# artist_table.to_csv('./data/artstore_match_v4_artist.csv', index=None)
artist_table.tail(3)

,name,group,no,phone,email,instagram,homepage,etc
37,전은미,M,1,010-4471-5995,multieunmi@naver.com,,,
38,나혜지,M,2,010-5553-8404,thedesignlab@naver.com,,,
39,정효윤,M,3,010-8200-7607,hony0709@naver.com,,,


In [6]:
from data.arterior import Arterior
table = Arterior().naverAddressTableInsert(table,'address',11)
table[table["lon"]==""]

100%|██████████| 148/148 [00:16<00:00,  8.76it/s]


,group,artist_no,artist,index,fix,view,title,type,ceo,phone,address,addressfull,addressroad,lon,lat


In [7]:
# 겹치는 주솟값 보정하기
from collections import Counter
_count_address = dict(Counter([_ for _ in table['addressfull']]))
_count_address = {k:str(v) for k, v in _count_address.items() if v != 1}
_count_address

{'서울특별시 관악구 난향길 14': '4',
 '서울특별시 관악구 난향길 12': '2',
 '서울특별시 관악구 난곡로24길 29': '3',
 '서울특별시 관악구 난곡로24길 6': '3',
 '서울특별시 관악구 난곡로26길 7': '2',
 '서울특별시 관악구 호암로22길 69 신국고시원': '2',
 '서울특별시 관악구 대학5길 21': '2',
 '서울특별시 관악구 신림로11길 20': '2',
 '서울특별시 관악구 신림로65길 31': '2',
 '서울특별시 관악구 남부순환로214길 29': '2',
 '서울특별시 관악구 봉천로 465': '2'}

In [13]:
# 중복 주소의 lon, lat 데이터 약간씩 수정
for _list in [list(table[table["addressfull"]==_address].index)  for _address in _count_address]:
    for no, index in enumerate(_list):
        lon, lat = table.loc[index, 'lon'], table.loc[index, 'lat']
        table.loc[index, 'lon'] = float(lon) - 0.000039 * no
        table.loc[index, 'lat'] = float(lat) - 0.000039 * no * 1.7

# 겹치는 주솟값 보정내용 저장하기
# table.to_csv('./data/artstore_match_v4_store.csv', index=None)

## **3 Editing the DATABASE**
위에서 생성한 자료 정리하기

In [22]:
import pandas as pd
file_artist = './data/artstore_match_v4_artist.csv'
file_store = './data/artstore_match_v4_store.csv'
df_artist = pd.read_csv(file_artist).fillna('')
df_store  = pd.read_csv(file_store).fillna('')
df_artist.tail(3)

,name,group,no,phone,email,instagram,homepage,etc
37,전은미,M,1,010-4471-5995,multieunmi@naver.com,,,
38,나혜지,M,2,010-5553-8404,thedesignlab@naver.com,,,
39,정효윤,M,3,010-8200-7607,hony0709@naver.com,,,


In [21]:
df_store.tail(3)

,group,artist_no,artist,index,fix,view,title,type,ceo,phone,address,addressfull,addressroad,lon,lat
145,C,12,임우현,C-12-2,0,1,신현선무관,학원,안현석,010-2275-5616,"장군봉1길 45, 2층",서울특별시 관악구 장군봉1길 45,서울특별시 관악구 봉천동 927-18,126.942606,37.481743
146,C,12,임우현,C-12-3,0,1,임서하헤어마벨,미용업,임서하,010-8689-8650,장군봉1길 29,서울특별시 관악구 장군봉1길 29,서울특별시 관악구 봉천동 930-3,126.941729,37.481842
147,C,12,임우현,C-12-4,0,1,정가,음식점,유영옥,010-5331-7715,남부순환로 1710,서울특별시 관악구 남부순환로 1710 이너스빌딩,서울특별시 관악구 봉천동 945-10 이너스빌딩,126.939874,37.482753
